In [1]:
pip install duckdb

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
    --------------------------------------- 0.3/12.3 MB ? eta -:--:--
    --------------------------------------- 0.3/12.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.3 MB 762.0 kB/s eta 0:00:16
   -- ------------------------------------- 0.8/12.3 MB 1.0 MB/s eta 0:00:12
   --- ------------------------------------ 1.0/12.3 MB 1.1 MB/s eta 0:00:11
   ---- ----------------------------------- 1.3/12.3 MB 1.0 MB/s eta 0:00:11
   ----- ---------------------------------- 1.6/12.3 MB 1.1 MB/s eta 0:00:10
   ----- ---------------------------------- 1.8/12.3 MB 1.1 MB/s eta 0:00:10
   ------ --------------------------------- 2.1/12.3 MB 1.1 MB/s eta 0:00:10
   ------- ----------------------------

In [2]:
import duckdb
import pandas as pd

In [3]:
file_path = "Future_Truck_Forecast_With_Predictions.xlsx"

df = pd.read_excel(file_path)
df.head()


,Date,Hour,Commodity,Dry_Cargo_Type,Operation,Customer,Storage_Type,Declared_Qty_Tons,Truck_Capacity_Tons,Gate_No,Gate_Lanes,Gate_Capacity_Trucks_Per_Hour,Peak_Hour_Flag,Predicted_Truck_Count,Gate_Utilization,Congestion_Index,Traffic_Level,Traffic_Level_Num
0,2025-11-27,6,Machinery,Break Bulk,Export,Tata Steel,Direct,148,14,Gate-C,2,35,0,4.055,0.115857,0.115857,Low,0
1,2025-07-05,0,Iron Ore,Dry Bulk,Export,FCI,Direct,1968,24,Gate-B,3,50,0,66.565,1.331300,1.331300,High,2
2,2025-08-15,12,Fertilizer,Dry Bulk,Import,NTPC,Storage,402,22,Gate-A,4,70,0,15.495,0.221357,0.521357,Low,0
3,2026-03-11,5,Steel Plates,Break Bulk,Import,Adani,Direct,149,12,Gate-C,2,35,0,3.685,0.105286,0.105286,Low,0
4,2025-10-06,10,Coal,Dry Bulk,Import,JSW,Storage,506,23,Gate-C,2,35,1,19.540,0.558286,1.058286,Medium,1


In [4]:
con = duckdb.connect()
con.register("gate_forecast", df)


TRAFFIC LEVEL DISTRIBUTION

In [5]:
con.execute("""
SELECT
    Traffic_Level,
    COUNT(*) AS total_hours,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM gate_forecast
GROUP BY Traffic_Level
ORDER BY total_hours DESC
""").df()


,Traffic_Level,total_hours,percentage
0,High,1947,48.68
1,Low,1499,37.48
2,Medium,554,13.85


GATEWISE CONGESTION RANKING 

In [6]:
con.execute("""
SELECT
    Gate_No,
    ROUND(AVG(Congestion_Index), 2) AS avg_congestion,
    ROUND(AVG(Gate_Utilization), 2) AS avg_utilization
FROM gate_forecast
GROUP BY Gate_No
ORDER BY avg_congestion DESC
""").df()


,Gate_No,avg_congestion,avg_utilization
0,Gate-D,1.68,1.43
1,Gate-C,1.44,1.20
2,Gate-B,1.11,0.86
3,Gate-A,0.81,0.58


HIGH TRAFFIC HOURS BY GATE 

In [8]:
con.execute("""
SELECT
    Gate_No,
    COUNT(*) AS high_traffic_hours
FROM gate_forecast
WHERE Traffic_Level = 'High'
GROUP BY Gate_No
ORDER BY high_traffic_hours DESC
""").df()


,Gate_No,high_traffic_hours
0,Gate-D,654
1,Gate-C,587
2,Gate-B,451
3,Gate-A,255


PEAK HOUR RISK ANALYSIS

In [10]:
con.execute("""
SELECT
    Hour,
    COUNT(*) AS high_traffic_events
FROM gate_forecast
WHERE Traffic_Level = 'High'
GROUP BY Hour
ORDER BY high_traffic_events DESC
""").df()


,Hour,high_traffic_events
0,16,101
1,19,98
2,7,91
3,15,88
4,9,88
5,6,86
6,10,86
7,8,85
8,11,85
9,18,84


DRY BULK VS BREAK BULK IMPACT 

In [11]:
con.execute("""
SELECT
    Dry_Cargo_Type,
    ROUND(AVG(Predicted_Truck_Count), 2) AS avg_trucks,
    ROUND(AVG(Congestion_Index), 2) AS avg_congestion
FROM gate_forecast
GROUP BY Dry_Cargo_Type
""").df()


,Dry_Cargo_Type,avg_trucks,avg_congestion
0,Break Bulk,4.55,0.27
1,Dry Bulk,58.61,1.71


IMPORT VS EXPORT RISK

In [13]:
con.execute("""
SELECT
    Operation,
    ROUND(AVG(Predicted_Truck_Count), 2) AS avg_trucks,
    ROUND(AVG(Congestion_Index), 2) AS avg_congestion
FROM gate_forecast
GROUP BY Operation
""").df()


,Operation,avg_trucks,avg_congestion
0,Import,41.16,1.31
1,Export,43.24,1.21


SLA BREACH FORECAST

In [14]:
con.execute("""
SELECT
    Date,
    Hour,
    Gate_No,
    Predicted_Truck_Count,
    Gate_Capacity_Trucks_Per_Hour,
    Congestion_Index
FROM gate_forecast
WHERE Congestion_Index > 1.5
  AND Predicted_Truck_Count > Gate_Capacity_Trucks_Per_Hour
ORDER BY Congestion_Index DESC
""").df()


,Date,Hour,Gate_No,Predicted_Truck_Count,Gate_Capacity_Trucks_Per_Hour,Congestion_Index
0,2025-12-17,7,Gate-D,121.460,30,4.548667
1,2025-10-29,8,Gate-D,119.795,30,4.493167
2,2025-12-10,8,Gate-D,113.960,30,4.298667
3,2026-03-01,19,Gate-D,113.445,30,4.281500
4,2026-02-28,8,Gate-D,110.040,30,4.168000
...,...,...,...,...,...,...
1472,2025-10-26,8,Gate-A,70.250,70,1.503571
1473,2025-12-02,14,Gate-B,60.115,50,1.502300
1474,2026-01-15,19,Gate-C,35.070,35,1.502000
1475,2025-07-05,18,Gate-D,30.050,30,1.501667


CROSS GATE LOAD SHFTING OPP

In [15]:
con.execute("""
SELECT
    a.Date,
    a.Hour,
    a.Gate_No AS congested_gate,
    b.Gate_No AS underutilized_gate
FROM gate_forecast a
JOIN gate_forecast b
  ON a.Date = b.Date
 AND a.Hour = b.Hour
WHERE a.Traffic_Level = 'High'
  AND b.Gate_Utilization < 0.5
  AND a.Gate_No <> b.Gate_No
""").df()


,Date,Hour,congested_gate,underutilized_gate
0,2026-01-01,2,Gate-C,Gate-A
1,2026-01-15,9,Gate-A,Gate-B
2,2025-07-26,9,Gate-C,Gate-A
3,2025-09-25,8,Gate-B,Gate-D
4,2026-01-02,14,Gate-C,Gate-B
...,...,...,...,...
373,2025-07-14,1,Gate-D,Gate-B
374,2025-10-14,16,Gate-C,Gate-A
375,2025-09-26,22,Gate-C,Gate-A
376,2026-03-30,18,Gate-A,Gate-D


TOP 10 HIGHEST CONGESTION EVENTS

In [16]:
con.execute("""
SELECT
    Date,
    Hour,
    Gate_No,
    Dry_Cargo_Type,
    Operation,
    Predicted_Truck_Count,
    Congestion_Index
FROM gate_forecast
ORDER BY Congestion_Index DESC
LIMIT 10
""").df()


,Date,Hour,Gate_No,Dry_Cargo_Type,Operation,Predicted_Truck_Count,Congestion_Index
0,2025-12-17,7,Gate-D,Dry Bulk,Export,121.460,4.548667
1,2025-10-29,8,Gate-D,Dry Bulk,Import,119.795,4.493167
2,2025-12-10,8,Gate-D,Dry Bulk,Import,113.960,4.298667
3,2026-03-01,19,Gate-D,Dry Bulk,Import,113.445,4.281500
4,2026-02-28,8,Gate-D,Dry Bulk,Export,110.040,4.168000
5,2025-11-15,9,Gate-D,Dry Bulk,Import,109.235,4.141167
6,2025-09-12,14,Gate-D,Dry Bulk,Export,114.995,4.133167
7,2025-11-11,17,Gate-D,Dry Bulk,Import,108.970,4.132333
8,2025-07-21,19,Gate-D,Dry Bulk,Import,106.910,4.063667
9,2026-03-02,13,Gate-D,Dry Bulk,Import,112.290,4.043000
